In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.functional as F
from torch.utils.data import Dataset
import torch
import h5py

In [2]:
with h5py.File('session/data_S1_Session1_S1_Session_1.h5', 'r') as f:
    eeg = np.array(f['data'])
    # So that the indexing works
    labels = np.array(f['label']) - 1

In [3]:
batch_size = 32
max_number_of_batches = eeg.shape[0] // batch_size
max_no_data = max_number_of_batches * batch_size

eeg_batches = eeg[:max_no_data]
labels_batches = labels[:max_no_data]

In [4]:
eeg_batches = eeg_batches.reshape(max_number_of_batches, batch_size, 1, 62, 500)
labels_batches = labels_batches.reshape(max_number_of_batches, batch_size)

In [5]:
print(eeg_batches.shape)
print(labels_batches.shape)

(1102, 32, 1, 62, 500)
(1102, 32)


In [6]:
rng = np.random.default_rng()
index = rng.choice(max_number_of_batches, size=max_number_of_batches, replace=False)

In [7]:
train_size = 800
validation_size = 200

# Test is not used

train_eeg = eeg_batches[index[:train_size]]
validation_eeg = eeg_batches[index[train_size:train_size+validation_size]]
test_eeg = eeg_batches[index[train_size+validation_size:]]

train_labels = labels_batches[index[:train_size]]
validation_labels = labels_batches[index[train_size:train_size+validation_size]]
test_labels = labels_batches[index[train_size+validation_size:]]

In [8]:
class DataLoader(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.Tensor(self.data[idx]), torch.LongTensor(self.labels[idx])

In [9]:
train_loader = DataLoader(train_eeg, train_labels)
validation_loader = DataLoader(validation_eeg, validation_labels)
test_loader = DataLoader(test_eeg, test_labels)

In [10]:
# Temporal Convolutional Network block (Residual Block)
class TCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, dropout):
        """
        Args:
        in_channels: number of channels
        out_channels: number of output channels
        kernel_size: the second parameter of the size of the kernel (because the first parameter needs to be one)
        stride: stride
        Padding - padding used for conv2d (dependant on the dilation)
        Dilation - parameter used for conv2d, it determines when we end adding new blocks to our algorithm
        Dropout - how many neurons should we drop
        """
        super().__init__()
        # 1. Padding on the left side
        pad = torch.nn.ZeroPad2d((padding, 0, 0, 0))
        # 2. Convolutional network (models usually include weight_norm)
        conv2d1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(1, kernel_size), stride=stride, padding=0, dilation=dilation)
        # 3. ELU
        elu = nn.ELU()
        # 4. Dropout
        dropout = nn.Dropout(dropout)
        # 5. Add one more layer (only conv2d is need because the rest doesn't contain weights)
        conv2d2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=(1, kernel_size), stride=stride, padding=0, dilation=dilation)
        # 6. Put everything in one place
        self.net = nn.Sequential(pad, conv2d1, elu, dropout,
                                 pad, conv2d2, elu, dropout)
        
    def forward(self, x):
        return self.net(x)

In [11]:
# Shallow ConvNet (don't mistake with residual ConvNet)
class ShallowNetwork(nn.Module):
    def __init__(self, in_channels, out_features, kernel_size, dropout):
        super().__init__()
        dilation = 1
        tcn1 = TCNBlock(in_channels=in_channels, out_channels=40, kernel_size=kernel_size, stride=1, padding=(kernel_size-1) * dilation, dilation=1, dropout=0.1)
        spatial_filter_1 = nn.Conv2d(in_channels=40, out_channels=40, kernel_size=(62, 1))
        batch_norm = nn.BatchNorm2d(num_features=40)
        elu = nn.ELU()
        avg2dpool1 = nn.AvgPool2d(kernel_size=(1, 56), stride=(1, 14))
        flatten = nn.Flatten(start_dim=1)
        linear1 = nn.Linear(in_features=1280, out_features=out_features)
#         softmax = nn.Softmax(dim=1)
        self.model = nn.Sequential(tcn1, spatial_filter_1, batch_norm, elu, avg2dpool1, flatten, linear1, softmax)
        
    def forward(self, x):
        return self.model(x)

In [12]:
torch.manual_seed(0)
model = ShallowNetwork(in_channels=1, out_features=4, kernel_size=26, dropout=0.1)
model

ShallowNetwork(
  (model): Sequential(
    (0): TCNBlock(
      (net): Sequential(
        (0): ZeroPad2d(padding=(25, 0, 0, 0), value=0.0)
        (1): Conv2d(1, 40, kernel_size=(1, 26), stride=(1, 1))
        (2): ELU(alpha=1.0)
        (3): Dropout(p=0.1, inplace=False)
        (4): ZeroPad2d(padding=(25, 0, 0, 0), value=0.0)
        (5): Conv2d(40, 40, kernel_size=(1, 26), stride=(1, 1))
        (6): ELU(alpha=1.0)
        (7): Dropout(p=0.1, inplace=False)
      )
    )
    (1): Conv2d(40, 40, kernel_size=(62, 1), stride=(1, 1))
    (2): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ELU(alpha=1.0)
    (4): AvgPool2d(kernel_size=(1, 56), stride=(1, 14), padding=0)
    (5): Flatten(start_dim=1, end_dim=-1)
    (6): Linear(in_features=1280, out_features=4, bias=True)
    (7): Softmax(dim=1)
  )
)

In [13]:
ov_no_params = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        ov_no_params += param.numel()
        print("{name}: no of params: {no_param}".format(name=name, no_param=param.numel()))
        
print("\nOverall amount of parameters: " + str(ov_no_params))

model.0.net.1.weight: no of params: 1040
model.0.net.1.bias: no of params: 40
model.0.net.5.weight: no of params: 41600
model.0.net.5.bias: no of params: 40
model.1.weight: no of params: 99200
model.1.bias: no of params: 40
model.2.weight: no of params: 40
model.2.bias: no of params: 40
model.6.weight: no of params: 5120
model.6.bias: no of params: 4

Overall amount of parameters: 147164


In [14]:
#raise NotImplementedError("Stop here")

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time

start_time = time.time()
no_epochs = 5
for epoch in range(no_epochs):
    train_loss = 0
    
    # Here normally you should use data loader but I don't want right now to define data loaders
    for i, train_data in enumerate(train_loader):
        eeg_inputs, eeg_labels = train_data
        optimizer.zero_grad()
        eeg_prediction = model(eeg_inputs)
        loss = criterion(eeg_prediction, eeg_labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        if i % 20 == 0:
            end_time = time.time()
            print("We are alive and we are at the epoch: {epoch} and at the point: {i} with time: {time}".format(epoch=epoch+1, i=i, time=end_time-start_time))
            start_time = end_time
        
    print("Train loss on epoch: {epoch} : {train_loss}".format(epoch=epoch+1, train_loss=train_loss))

We are alive and we are at the epoch: 1 and at the point: 0 with time: 5.49254035949707
We are alive and we are at the epoch: 1 and at the point: 20 with time: 78.39482307434082
We are alive and we are at the epoch: 1 and at the point: 40 with time: 80.65927386283875
We are alive and we are at the epoch: 1 and at the point: 60 with time: 76.90121555328369
We are alive and we are at the epoch: 1 and at the point: 80 with time: 75.97425031661987
We are alive and we are at the epoch: 1 and at the point: 100 with time: 74.57956004142761
We are alive and we are at the epoch: 1 and at the point: 120 with time: 75.18793034553528
We are alive and we are at the epoch: 1 and at the point: 140 with time: 73.59290337562561
We are alive and we are at the epoch: 1 and at the point: 160 with time: 78.58671736717224
We are alive and we are at the epoch: 1 and at the point: 180 with time: 78.45951199531555
We are alive and we are at the epoch: 1 and at the point: 200 with time: 76.60233426094055
We are

We are alive and we are at the epoch: 3 and at the point: 200 with time: 104.35272312164307
We are alive and we are at the epoch: 3 and at the point: 220 with time: 105.73270559310913
We are alive and we are at the epoch: 3 and at the point: 240 with time: 107.23386359214783
We are alive and we are at the epoch: 3 and at the point: 260 with time: 107.56246089935303
We are alive and we are at the epoch: 3 and at the point: 280 with time: 114.0200743675232
We are alive and we are at the epoch: 3 and at the point: 300 with time: 114.16831254959106
We are alive and we are at the epoch: 3 and at the point: 320 with time: 120.9426658153534
We are alive and we are at the epoch: 3 and at the point: 340 with time: 118.26021265983582
We are alive and we are at the epoch: 3 and at the point: 360 with time: 121.17269468307495
We are alive and we are at the epoch: 3 and at the point: 380 with time: 124.66216325759888
We are alive and we are at the epoch: 3 and at the point: 400 with time: 126.02198

In [ ]:
# You should use validation in the training but we will use it here to make it easier
overall_accuracy_validation = 0
for i, val_data in enumerate(validation_loader):
    eeg_inputs, eeg_labels = val_data
    
    with torch.no_grad():
        eeg_prediction = torch.argmax(model(eeg_inputs))
        
    accuracy = (eeg_labels == eeg_prediction).sum().item() / batch_size
    overall_accuracy_validation += accuracy
    
    print("Accuracy for batch no {i}: {accuracy}".format(i=i, accuracy=accuracy))

print("Overall accuracy: ", accuracy/len(validation_loader))